# Подгрузка пакетов и данных

In [ ]:
cd ..

In [ ]:
import json

from gensim.models.word2vec import Word2Vec

Загрузим уже построенную модель Word2Vec

In [ ]:
word2VecModel = Word2Vec.load('word2Vec/word2VecModel')

Загрузим JSON, в котором мы сложили названия функций (и названия их арггументов)

In [ ]:
path_to_functions_json = 'data/ast-functions.json'
  
functions_storage_dictionary = json.load(open(path_to_functions_json))

PROPERTY_WITH_FUNC_NAME = 'functionName' # под каким ключом в json складывали название функции
PROPERTY_WITH_ARGS_NAMES = 'argumentsNames' # под каким ключом в json складывали названия аргументов функции
PROPERTY_WITH_FUNC_TYPE = 'type' # под каким ключом в json складывали тип функции

# Отделение объявления функций от их использования

In [ ]:
def find_all_functions_creations(func_storage_dic):
    func_creation_dic = {}
    
    for fileName in func_storage_dic.keys():
        for function in func_storage_dic[fileName]:
            if function[PROPERTY_WITH_FUNC_TYPE] == 'FunctionDeclaration' or function[PROPERTY_WITH_FUNC_TYPE] == 'FunctionExpression':
                if function[PROPERTY_WITH_FUNC_NAME] in func_creation_dic:
                    print(function[PROPERTY_WITH_FUNC_NAME], 'уже есть в словаре')
                func_creation_dic[function[PROPERTY_WITH_FUNC_NAME]] = function[PROPERTY_WITH_ARGS_NAMES]
    
    return func_creation_dic

In [ ]:
func_creation_dic = find_all_functions_creations(functions_storage_dictionary)

А теперь находим все ВЫЗОВЫ функций и смотрим, насколько их аргументы отличны объявления функции

In [ ]:
def find_all_func_executions(func_storage_dic, func_creation_dic):
    func_exec_storage = []
    
    for fileName in func_storage_dic.keys():
        for function in func_storage_dic[fileName]:
            if (
                function[PROPERTY_WITH_FUNC_TYPE] == 'CallExpression' and # должна быть вызовом функции
                len(function[PROPERTY_WITH_ARGS_NAMES]) > 0 and # должна иметь хоть один аргумент
                function[PROPERTY_WITH_FUNC_NAME] in func_creation_dic # должна быть в нашем словаре
            ):
                func_exec_storage.append(function)
    
    return func_exec_storage

In [ ]:
func_exec_storage = find_all_func_executions(functions_storage_dictionary, func_creation_dic)

In [ ]:
print('Размер словаря с объявлениями функций: ', len(func_creation_dic))
print('Размер массива с вызовами функций: ', len(func_exec_storage))

Словарь с объявлениями функций - словарь, в котором ключ - название функции, а значение – массив с названиями переданных аргументов.

Пример: функция с названием **confirmDelete**

In [ ]:
func_creation_dic['confirmDelete']

Массив с вызовами функций состоит из словарей следующего формата

In [ ]:
func_exec_storage[0]

# Алгоритм проверки схожести аргументов

In [ ]:
def checkIfWordInWord2VecDict(word, word2VecVocabulary = word2VecModel.wv):
    return word in word2VecVocabulary

def get_args_similarity(exec_func_name, exec_func_args_names , func_dict, word2VecModel):
    """Определяет схожесть названий аргументов в вызове функции с названиями аргументов в объявлении функции"""
    args_similarity = []
    
    for argIndex in range(len(exec_func_args_names)):
        if argIndex > len(func_dict[exec_func_name]) - 1:
            args_similarity.append(None)
            continue
        
        exec_arg = exec_func_args_names[argIndex]
        create_arg = func_dict[exec_func_name][argIndex]
        
        if (checkIfWordInWord2VecDict(exec_arg) and checkIfWordInWord2VecDict(create_arg)):
            args_similarity.append(word2VecModel.wv.similarity(exec_arg, create_arg))
        else:
            args_similarity.append(None)
        
    return args_similarity

Посмотрим, как это работает на примере

In [ ]:
example_func_exec = func_exec_storage[-2]

func_name = example_func_exec[PROPERTY_WITH_FUNC_NAME]
args_names = example_func_exec[PROPERTY_WITH_ARGS_NAMES]


print('Вызов функции:', func_name, 'c аргументами', args_names)
print('Функция же была объявлена с словаре с аргументами:', func_creation_dic[func_name])

get_args_similarity(func_name, args_names, func_creation_dic, word2VecModel)

# Реализация на всем массиве вызовов функции

Идея максимально проста. Проходим по массиву вызовов функции.

- По названию функции определяем в словаре с объявлениями функций, с какими названиями аргументов была объявлена эта функция
- Для каждой пары аргументов (название аргумента в момент вызовы функции и название аргумента в момент объявление функции) считаем близость их названий в векторной форме с помощью уже построенной модели word2Vec (это будет значение 0 до 1)
- Если эта схожесть меньше заданного порога (threshold), то выбрасываем предупреждение пользователю

### Treshold

In [ ]:
MIN_SIMILARITY_THRESHOLD = 0.2

Порог не должен быть большой, потому что модели важно достичь высокого precision, а не recall.

Если наш алгоритм будет часто беспокоить пользователя по пустякам, предупреждая об ошибках там, где их нет (false positive), то нашим алгоритмом просто никто не будет пользоваться. Нужно беспокоить программиста только прям в самых вероятных случаях наличия ошибки. Ведь программисты люди ленивые и занятые... 

In [ ]:
for func_exec in func_exec_storage:
    func_name = func_exec[PROPERTY_WITH_FUNC_NAME]
    args_names = func_exec[PROPERTY_WITH_ARGS_NAMES]
    
    args_similarities = get_args_similarity(func_name, args_names, func_creation_dic, word2VecModel)
    is_below_threshold_args_similarities = list(map(lambda x: x and x < MIN_SIMILARITY_THRESHOLD, args_similarities))
    
    if (any(is_below_threshold_args_similarities)):
        print(func_name, args_names)